In [2]:
from docx import Document

def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

doc1 = extract_text_from_docx('./docs/1747395490801_PMT Pro User FAQ.docx')
doc2 = extract_text_from_docx('./docs/1747395531476_Status and Evaluation on 15MAY2025 - PM Pro.docx')

raw_knowledge = doc1 + "\n" + doc2
len(raw_knowledge)

34442

In [2]:
import textwrap

def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(raw_knowledge)
len(chunks)

21

In [3]:
import requests

def get_ollama_embedding(text, model="mxbai-embed-large"):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": model, "prompt": text}
    )
    return response.json()["embedding"]

chunk_embeddings = [get_ollama_embedding(chunk) for chunk in chunks]

In [4]:
len(chunk_embeddings[0])

1024

In [5]:
len(chunk_embeddings)

21

In [6]:
import chromadb

# Use PersistentClient in v0.4+
chroma_client = chromadb.PersistentClient(path="./rag_chroma")
collection = chroma_client.get_or_create_collection("pmt_pro_docs")

for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
    collection.add(
        documents=[chunk],
        embeddings=[embedding],
        ids=[f"chunk_{i}"]
    )

In [7]:
def get_query_embedding(query):
    return get_ollama_embedding(query)

def retrieve_context(query, top_k=3):
    query_embedding = get_query_embedding(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return "\n---\n".join(results['documents'][0])

query = "How does PMT Pro handle overdue tasks?"
retrieved_context = retrieve_context(query)
print(retrieved_context)

PMT Pro - User FAQ (Frequently Asked Questions) GENERAL USAGE Q1: What is PMT Pro? A1: PMT Pro is an AI-powered project management tool designed to help teams plan, collaborate, and execute projects more efficiently. It integrates traditional project management features like Kanban boards, task lists, timelines, and team chat with advanced AI capabilities including smart task prioritization, predictive analytics, and an expert chatbot assistant, along with on-tip-availability of everything about the project using Agentic RAG technology for keeping all informed with relevant information, for the ask. Q2: How is PMT Pro different from Basecamp, Trello, or Asana? A2: Unlike those platforms, PMT Pro includes: A 24/7 AI assistant powered by a Retrieval-Augmented Generation (RAG) model Automatic task prioritization and deadline tracking Dynamic filtering by project context Meeting summaries, intelligent suggestions, and risk alerts A built-in document and link repository for each project Q3:

In [8]:
def generate_answer(query, context):
    prompt = f"""You are a helpful assistant for PMT Pro. Use the context below to answer the user's question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "llama3.1:8b", "prompt": prompt, "stream": False}
    )
    return response.json()["response"]

answer = generate_answer(query, retrieved_context)
print(answer)

According to Question 7 (A7) in the PMT Pro User FAQ, "The To-Do list is automatically sorted by due date, with overdue tasks highlighted at the top." This means that PMT Pro will highlight overdue tasks at the top of the To-Do list, making it easy for team members and project managers to see which tasks are past their deadline.


In [9]:
from IPython.display import Markdown, display

In [10]:
display(Markdown(answer))

According to Question 7 (A7) in the PMT Pro User FAQ, "The To-Do list is automatically sorted by due date, with overdue tasks highlighted at the top." This means that PMT Pro will highlight overdue tasks at the top of the To-Do list, making it easy for team members and project managers to see which tasks are past their deadline.

In [11]:
query = "How do I use the Kanban board?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

According to our FAQ (Q4), "How do you use the Kanban board?" is answered as follows:

"The Kanban board displays tasks in four stages: To Do, In Progress, Review, and Done. You can drag and drop cards between stages, assign due dates, and add team members. Each card can also include notes and priority tags."

In simpler terms, to use the Kanban board, you can follow these steps:

1. Create or move tasks to one of the four stages (To Do, In Progress, Review, Done).
2. Assign due dates to tasks.
3. Add team members to tasks.
4. Use notes and priority tags on each card.

Let me know if you have any further questions!

In [12]:
query = "How does team chat work in PMT Pro?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Team chat in PMT Pro is integrated with project data, allowing the AI to reference chat context when providing answers or suggestions. Team conversations can be quickly turned into linked tasks or knowledge entries. This means that chats are not just a communication channel, but also closely tied to work tracking and project management.

In more detail:

* The team chat is a collaborative space where team members can communicate with each other in real-time.
* The AI assistant can reference the context of the conversation when providing answers or suggestions.
* Team conversations can be quickly turned into linked tasks or knowledge entries, allowing for seamless transition from discussion to action.
* This integration enables teams to have contextual and productive discussions that are directly connected to their project work.

Overall, team chat in PMT Pro is designed to facilitate collaboration, productivity, and effective communication among team members.

In [13]:
query = "Can the chatbot be customized or trained further?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Yes. The chatbot continuously improves as more documents and chat context are added. Future updates will support custom project goals, style preferences, and voice tuning.

In [14]:
query = "What if I find a bug or have a feature request?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Use the chatbot or contact your admin to report issues or make suggestions. We welcome your feedback to continue improving PMT Pro.

In [15]:
query = "List the applications that PMT Pro can handle."
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Based on the provided context, here are some applications that PMT Pro can handle:

1. **Project planning**: PMT Pro helps teams plan projects with features like Kanban boards, task lists, timelines, and team chat.
2. **Task management**: The platform enables smart task prioritization, deadline tracking, and dynamic filtering by project context.
3. **Document storage and sharing**: PMT Pro has a built-in document and link repository for each project, allowing teams to store and share important files and links.
4. **Communication and collaboration**: The platform includes team chat, meeting summaries, intelligent suggestions, and risk alerts to facilitate real-time collaboration.
5. **Agile software development**: PMT Pro is ideal for agile software teams, consultancies, distributed teams, and organizations managing multiple concurrent projects.

These features indicate that PMT Pro can handle a wide range of applications related to project management, task management, communication, and document sharing.

In [16]:
query = "How do we prioritize the task on PMT?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

According to our system, PMT Pro uses AI-powered task prioritization to automatically rank and highlight tasks based on urgency, deadlines, and dependencies. This means that the system will analyze project data and user behavior to recommend what each user should focus on next.

To prioritize tasks on PMT, you can follow these steps:

1. Go to the "Tasks" page.
2. The system will automatically sort tasks by due date, with overdue tasks highlighted at the top.
3. You can also use the color-coded priority labels:
	* Red: High priority
	* Orange: Medium priority
	* Green: Low priority
4. Additionally, you can ask the AI assistant for help prioritizing your tasks. The chatbot can analyze project data and provide personalized recommendations on what to focus on next.

By using these features, PMT Pro's AI-powered task prioritization helps ensure that you and your team members are always working on the most critical tasks first.

In [17]:
query = "Give a complete overview of this PMT. What it is for and what it isn't for? Give the response in bullet points."
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Here's a comprehensive overview of PMT Pro:

**What PMT Pro Is For:**

* An AI-powered project management tool designed to help teams plan, collaborate, and execute projects more efficiently
* Integrates traditional project management features with advanced AI capabilities for smart task prioritization, predictive analytics, and expert chatbot assistance
* Ideal for agile software teams, consultancies, distributed teams, and organizations managing multiple concurrent projects
* Designed for simplicity and power, suitable for both daily users and technical/project leads

**Key Features:**

* 24/7 AI assistant powered by a Retrieval-Augmented Generation (RAG) model
* Automatic task prioritization and deadline tracking
* Dynamic filtering by project context
* Meeting summaries, intelligent suggestions, and risk alerts
* Built-in document and link repository for each project
* Kanban board with four stages: To Do, In Progress, Review, and Done
* Color-coded priority labels (Red: High priority, Orange: Medium priority, Green: Low priority)
* Tasks and Kanban cards, which can be assigned independently or tied to a process

**What PMT Pro Isn't For:**

* A simple, minimalistic tool (it's designed for mixed-skill teams with both power and simplicity)
* A platform that lacks advanced AI capabilities (it offers predictive analytics, smart task prioritization, and expert chatbot assistance)
* A solution that doesn't support real-time collaboration (it has built-in voice/video meeting integration and instant meeting links)
* A tool that doesn't centralize important files and knowledge (it has a "Important Files and Links" section with searchable content)

Overall, PMT Pro is designed to provide a comprehensive project management solution with advanced AI capabilities, making it an attractive option for teams looking for a powerful and user-friendly platform.